In [3]:
import timm
import time 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
# from medmnist import INFO
import numpy as np
import faiss
import copy
from tqdm import tqdm

from torch.nn.functional import softmax, cosine_similarity
from collections import Counter
import matplotlib.pyplot as plt
import torchvision.transforms.functional as TF
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import warnings
warnings.filterwarnings("ignore")

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print("Using device:", device)

Using device: cuda:0


In [5]:
import os
from torchvision import datasets, transforms
import torch

print(f"Step 1: Loading dataset with resize transform...")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    # transforms.Normalize([0.485, 0.456, 0.406],
    #                      [0.229, 0.224, 0.225]) 
])

val_data_dir = 'dataset/imagenet_validation' 
test_data_dir = 'dataset/imagenet_tests'

val_dataset = datasets.ImageFolder(os.path.join(val_data_dir), transform=transform) 

test_datasets = {
    f'test{i}': datasets.ImageFolder(os.path.join(test_data_dir, f'test{i}'), transform=transform)
    for i in range(1, 11)
}

print(f"Validation samples: {len(val_dataset)}")
print(f"Validation samples: {len(test_datasets['test1'])}")

Step 1: Loading dataset with resize transform...
Validation samples: 30000
Validation samples: 2004


In [6]:
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4) 
test_loader = DataLoader(test_datasets['test1'], batch_size=1, shuffle=False, num_workers=4)  

In [7]:
from torch.utils.data import Subset, DataLoader
import random
import torch

# Set random seeds for reproducibility
seed = 42
random.seed(seed)
torch.manual_seed(seed)

# Get total length
n_total = len(test_datasets['test1'])  # should be 2000

# Randomly choose 200 unique indices
subset_indices = random.sample(range(n_total), 200)

# Create subset
subset_test = Subset(test_datasets['test1'], subset_indices)

In [8]:
test_loader = DataLoader(subset_test, batch_size=1, shuffle=False, num_workers=4) 

### SVRE Attack

In [9]:
import torch
import torch.nn.functional as F
import random
from typing import List

def _model_forward_logits(model, x):
    """Forward pass that handles tuple outputs (e.g., inception)."""
    out = model(x)
    if isinstance(out, (tuple, list)):
        out = out[0]
    return out

def _grad_wrt_input(model, x, y, loss_fn=None):
    """
    Compute gradient of loss wrt x for a single model.
    x should require_grad=True and be a float tensor on device.
    Returns gradient tensor same shape as x (detached).
    """
    if loss_fn is None:
        loss_fn = torch.nn.CrossEntropyLoss(reduction='mean')
    x = x.clone().detach().requires_grad_(True)
    out = _model_forward_logits(model, x)
    loss = loss_fn(out, y)
    grad = torch.autograd.grad(loss, x, retain_graph=False, create_graph=False)[0]
    return grad.detach()

def svre_mi_fgsm(
    models: List[torch.nn.Module],
    x: torch.Tensor,
    y: torch.Tensor,
    eps: float = 8/255,
    T: int = 10,
    M: int = 5,
    beta: float = 1/255,       # inner step size
    mu1: float = 1.0,         # outer momentum decay (like MI-FGSM's decay)
    mu2: float = 1.0,         # inner momentum decay
    alpha: float = None,      # outer step size (if None set to eps/T)
    clip_min: float = 0.0,
    clip_max: float = 1.0,
    loss_fn = None,
    device_str: str = None,
    random_seed: int = None,
):
    """
    SVRE-MI-FGSM untargeted ensemble attack (L_inf).

    Args:
      models: list of surrogate models (already .eval() and on device).
      x: input batch [B,C,H,W] in [clip_min,clip_max]
      y: labels [B]
      eps: L_inf budget
      T: outer iterations (integer)
      M: inner update frequency (integer)
      beta: inner step size (L_inf scale)
      mu1: outer momentum decay (0..1). Typical MI-FGSM uses 1.0.
      mu2: inner momentum decay (0..1)
      alpha: outer step size. If None uses eps/T
      clip_min/clip_max: pixel bounds
      loss_fn: optional loss function
      device_str: optional device (e.g., "cuda:0"); otherwise uses x.device
      random_seed: optional for reproducibility
    Returns:
      x_adv: adversarial batch tensor (same device as x)
    """
    if device_str is None:
        device = x.device
    else:
        device = torch.device(device_str)

    if random_seed is not None:
        random.seed(random_seed)
        torch.manual_seed(random_seed)

    if loss_fn is None:
        loss_fn = torch.nn.CrossEntropyLoss(reduction='mean')

    if alpha is None:
        alpha = eps / float(T)

    # ensure models list
    if not isinstance(models, (list, tuple)):
        models = [models]

    # move models to device and eval
    for m in models:
        m.to(device)
        m.eval()

    x_orig = x.clone().detach().to(device)
    x_adv = x_orig.clone().detach()
    x_adv = x_adv.float()

    # momentum accumulators (same shape as x)
    G = torch.zeros_like(x_adv, device=device)  # outer momentum

    B = x_adv.shape[0]
    K = len(models)

    # Outer loop
    for t in range(T):
        # 1) Compute per-model gradients at current outer x_adv (grad_k(x_adv))
        grads_at_xadv = []
        for k, model in enumerate(models):
            gk = _grad_wrt_input(model, x_adv, y, loss_fn=loss_fn)  # shape [B,C,H,W]
            grads_at_xadv.append(gk)
        # convert to tensor [K, B, C, H, W] by stacking
        grads_stack = torch.stack(grads_at_xadv, dim=0)  # (K, B, C, H, W)
        # ensemble gradient = mean over K
        g_ens = grads_stack.mean(dim=0)  # shape [B,C,H,W]

        # 2) Inner loop: variance-reduced stochastic updates
        x_tilde = x_adv.clone().detach()
        G_tilde = torch.zeros_like(x_tilde, device=device)

        for m_idx in range(M):
            # sample a random model index k
            k = random.randrange(K)
            model_k = models[k]

            # compute grad at x_tilde for model k
            grad_k_xtilde = _grad_wrt_input(model_k, x_tilde, y, loss_fn=loss_fn)  # [B,C,H,W]

            # grad_k_xadv was precomputed:
            grad_k_xadv = grads_at_xadv[k]  # [B,C,H,W]

            # variance reduced gradient: g_tilde = grad_k_xtilde - (grad_k_xadv - g_ens)
            g_tilde = grad_k_xtilde - (grad_k_xadv - g_ens)

            # momentum update (inner)
            G_tilde = mu2 * G_tilde + g_tilde

            # inner sign-step update (L_inf style)
            x_tilde = x_tilde.detach() + beta * G_tilde.sign()
            # project inner to L_inf around x_adv (inner loop uses x_adv as reference per paper)
            delta_inner = torch.clamp(x_tilde - x_adv, min=-eps, max=eps)
            x_tilde = torch.clamp(x_adv + delta_inner, min=clip_min, max=clip_max).detach()

        # 3) After inner loop, update outer momentum and x_adv
        # Use accumulated inner momentum G_tilde (the last)
        G = mu1 * G + G_tilde  # outer momentum update
        x_adv = x_adv.detach() + alpha * G.sign()
        # project to L_inf ball around original x_orig and clip valid range
        delta = torch.clamp(x_adv - x_orig, min=-eps, max=eps)
        x_adv = torch.clamp(x_orig + delta, min=clip_min, max=clip_max).detach()

    return x_adv


### Prepare for Attack 

In [10]:
def get_models(dataset, model_name, key): 
    if dataset == 'imagenet':
        # save_root_path = r"checkpoint/tinyimagenet"
        model = timm.create_model(model_name, pretrained=True, num_classes=1000).to(device)
        model.eval()
        if 'inc' in key or 'vit' in key or 'bit' in key:
            return torch.nn.Sequential(transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), model)
        else:
            return torch.nn.Sequential(transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), model)

In [60]:
ens_models = [
    get_models("imagenet", "resnet18", "resnet18"), 
    get_models("imagenet", "deit_tiny_patch16_224", "deit_t"),
    get_models("imagenet", "inception_v3", "inc_v3"), 
    get_models("imagenet", "vit_tiny_patch16_224", "vit_t"), 
    # get_models("imagenet", "efficientnet_b0", "efficientnet_b0"), 
    # get_models("imagenet", "swin_tiny_patch4_window7_224", "swin_t"), 
    # get_models("imagenet", "xcit_tiny_12_p8_224", "swin_t"), 
    # get_models("imagenet", "regnetv_040", "swin_t"), 
] 

In [61]:
adv_images = []      
adv_labels = []

eps = 8/255
alpha = 2/255
iters = 10

for data, label in tqdm(test_loader):
    data, label = data.to(device), label.to(device)
    # generate adv on batch (works for batch_size > 1)
    adv = svre_mi_fgsm(ens_models, data, label, eps=eps, clip_min=0.0, clip_max=1.0)
    # detach and move to CPU to avoid OOM
    adv_images.append(adv.detach().cpu())
    adv_labels.append(label.detach().cpu())

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [10:48<00:00,  3.24s/it]


In [20]:
target_models = [
    get_models("imagenet", "resnet50", "resnet50"), 
    get_models("imagenet", "resnet152", "resnet152"),
    get_models("imagenet", "wide_resnet101_2", "wrn101_2"), 
    get_models("imagenet", "resnetv2_50x1_bitm", "bit50_1"),
    
    get_models("imagenet", "resnetv2_101x1_bitm", "bit101_1"),
    get_models("imagenet", "vit_base_patch16_224", "vit_b"),
    get_models("imagenet", "deit_base_patch16_224", "deit_b"),
    get_models("imagenet", "swin_base_patch4_window7_224", "swin_b"), 
    get_models("imagenet", "swin_small_patch4_window7_224", "swin_s")
] 

In [63]:
for model in target_models:
    model.eval().to(device)
    correct, total = 0, 0
    with torch.no_grad():
        for adv_img, label in tqdm(zip(adv_images, adv_labels), 
                                   total=len(adv_images),
                                   desc=f"Evaluating {model.__class__.__name__} on adv"):
            adv_img = adv_img.to(device)
            label = label.to(device)

            if len(adv_img.size()) == 3: 
                outputs = model(adv_img.unsqueeze(0))
            if len(adv_img.size()) == 4: 
                outputs = model(adv_img) 
                
            if isinstance(outputs, tuple):  # handle models like Inception
                outputs = outputs[0]
            
            preds = outputs.argmax(dim=1)
            correct += (preds == label).sum().item()
            total += label.size(0)
    
    acc = 100 * correct / total
    print(f"{model.__class__.__name__} Accuracy on adv images: {acc:.2f}%")

Evaluating Sequential on adv: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:01<00:00, 135.05it/s]


Sequential Accuracy on adv images: 40.50%


Evaluating Sequential on adv: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:03<00:00, 64.77it/s]


Sequential Accuracy on adv images: 48.50%


Evaluating Sequential on adv: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:02<00:00, 86.07it/s]


...

Sequential Accuracy on adv images: 29.00%


Evaluating Sequential on adv: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:03<00:00, 52.30it/s]


Sequential Accuracy on adv images: 65.00%


Evaluating Sequential on adv: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:03<00:00, 52.12it/s]

Sequential Accuracy on adv images: 53.50%


### DEAA 

In [11]:
class NormalizedModel(nn.Module):
    def __init__(self, model, mean, std):
        super().__init__()
        self.model = model
        self.register_buffer('mean', torch.tensor(mean).view(1,3,1,1))
        self.register_buffer('std', torch.tensor(std).view(1,3,1,1))

    def forward(self, x):
        x = (x - self.mean) / self.std
        return self.model(x)

def get_last_linear_layer(model):
    """
    Find the last Linear layer in timm models or wrapped models (NormalizedModel).
    """
    # unwrap NormalizedModel if needed
    if isinstance(model, NormalizedModel):
        model = model.model

    # Common attribute names for classifier heads
    candidate_attrs = ['head', 'heads', 'fc', 'classifier', 'mlp_head']

    for attr in candidate_attrs:
        if hasattr(model, attr):
            layer = getattr(model, attr)
            # If it's a Linear layer
            if isinstance(layer, nn.Linear):
                return layer
            # If it's a Sequential or Module, search inside
            if isinstance(layer, nn.Module):
                last_linear = None
                for m in reversed(list(layer.modules())):
                    if isinstance(m, nn.Linear):
                        last_linear = m
                        break
                if last_linear is not None:
                    return last_linear

    # Fallback: scan all modules
    last_linear = None
    for m in model.modules():
        if isinstance(m, nn.Linear):
            last_linear = m
    if last_linear is not None:
        return last_linear

    raise RuntimeError(f"No Linear layer found in model {model.__class__.__name__}")



def get_features_before_last_linear(model, x):
    """
    Extract features before the final classifier, works for CNNs and ViTs.
    """
    # unwrap NormalizedModel if present
    if isinstance(model, NormalizedModel):
        model = model.model

    # Common classifier attributes
    candidate_attrs = ['head', 'heads', 'fc', 'classifier', 'mlp_head']
    classifier = None
    for attr in candidate_attrs:
        if hasattr(model, attr):
            classifier = getattr(model, attr)
            break

    features = {}

    def hook(module, input, output):
        features['feat'] = input[0].detach()

    if classifier is not None:
        handle = classifier.register_forward_hook(hook)
    else:
        # fallback: attach hook to last module
        last_module = list(model.modules())[-1]
        handle = last_module.register_forward_hook(hook)

    model.eval()
    with torch.no_grad():
        _ = model(x)
    handle.remove()

    if 'feat' not in features:
        raise RuntimeError(f"Failed to capture features from model {model.__class__.__name__}")

    return features['feat']

In [12]:
def fire_check(local_labels, preds, per_class_min=1):
    local_labels = np.asarray(local_labels)

    # If preds are logits/probs, convert to labels
    preds = np.asarray(preds)
    if preds.ndim > 1:
        preds = preds.argmax(axis=1)

    # Classes present in the RoC (unique, not repeated)
    classes_in_roc = np.unique(local_labels)

    # Check: for each class c in RoC, there is at least `per_class_min` correct prediction
    missing = []
    for c in classes_in_roc:
        mask = (local_labels == c)
        n_correct = int(np.sum(preds[mask] == c))
        if n_correct < per_class_min:
            missing.append((int(c), n_correct))  # track which class is short

    fire_ok = (len(missing) == 0)
    return fire_ok

In [40]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader
import numpy as np
import faiss
from tqdm import tqdm
import torch.nn.functional as F


def softmax(x, dim=0):
    return F.softmax(x, dim=dim)


def cosine_similarity(x, y, dim=1, eps=1e-8):
    return F.cosine_similarity(x, y, dim=dim, eps=eps)


# 👀 Dummy visualization (replace with your function)
def visualize_test_and_roc(test_img, roc_imgs, local_labels):
    print("Visualization placeholder: Test image + RoC samples")
    print(f"RoC labels: {local_labels}")


class VisionDES_2: 
    def __init__(self, dsel_dataset, pool): 
        self.dsel_dataset = dsel_dataset
        self.dsel_loader = DataLoader(dsel_dataset, batch_size=32, shuffle=False) 
        self.dino_model = timm.create_model('vit_base_patch16_224.dino', pretrained=True).to(device)
        self.dino_model.eval()  
        self.pool = pool 

        self.suspected_model_votes = [] 
        
        
    def dino_embedder(self, images):
        if images.shape[1] == 1:
            images = images.repeat(1, 3, 1, 1)
        return self.dino_model.forward_features(images)


    def fit(self): 
        dsel_embeddings = []
        dsel_labels = []
    
        with torch.no_grad():
            for imgs, labels in tqdm(self.dsel_loader):
                imgs = imgs.to(device)
                embs = self.dino_embedder(imgs).cpu()  
                dsel_embeddings.append(embs)
                dsel_labels.append(labels)
    
        # Keep as tensor
        dsel_embeddings_tensor = torch.cat(dsel_embeddings).detach().cpu()  
        cls_tensor = dsel_embeddings_tensor[:, 0, :]  
    
        # Convert to NumPy
        cls_embeddings = np.ascontiguousarray(cls_tensor.numpy(), dtype='float32')
        self.dsel_embeddings = cls_embeddings
        self.dsel_labels = torch.cat(dsel_labels).numpy()
    
        # Build FAISS index
        embedding_dim = cls_embeddings.shape[1]
        self.index = faiss.IndexFlatL2(embedding_dim)
        self.index.add(cls_embeddings)


    def predict_weighted_robust(self, test_img, k=7, return_logits=False, explain=False, top=False, n=3, 
                                use_fire=False, per_class_min=1, use_sim=False, sim_threshold=0, 
                                alpha=0.6, knorae=False):
       # Step 1: Get DINO CLS embedding for the test image
        img_for_dino = test_img.unsqueeze(0).to(device)

        with torch.no_grad():
            test_emb = self.dino_model.forward_features(img_for_dino).cpu().numpy().astype('float32')
            test_emb = test_emb[:, 0, :]  # CLS token only
    
        # Step 2: Find k nearest neighbors in FAISS
        distances, neighbors = self.index.search(test_emb, k)
        neighbor_idxs = neighbors[0]
        local_labels = self.dsel_labels[neighbor_idxs]
        local_labels = np.array(local_labels).flatten()
    
        # Step 3: Get RoC images
        with torch.no_grad():
            roc_imgs = torch.stack([self.dsel_dataset[idx][0] for idx in neighbor_idxs]).to(device)
    
        # Step 4: Evaluate classifiers
        competences, soft_outputs, feature_similarities, passed_fire, correct_counts = [], [], [], [], []
    
        test_img_batch = test_img.unsqueeze(0).to(device)
    
        for clf in self.pool:
            clf.eval()
            with torch.no_grad():
                outputs = clf(roc_imgs)
                preds = outputs.argmax(dim=1).cpu().numpy()
                correct = (preds == local_labels).sum()
                competence = correct / k
                competences.append(competence)
                correct_counts.append(correct)

                # 🔥 FIRE check
                fire_ok = fire_check(local_labels, preds, per_class_min=per_class_min) 
                passed_fire.append(fire_ok)

                logits = clf(test_img.unsqueeze(0).to(device)).squeeze(0)
                probs = softmax(logits, dim=0)
                soft_outputs.append(probs)

                # Feature similarity
                test_feat = get_features_before_last_linear(clf, test_img.unsqueeze(0).to(device))
                roc_feats = get_features_before_last_linear(clf, roc_imgs)
                mean_feat = roc_feats.mean(dim=0, keepdim=True)

                test_vec = test_feat.flatten().unsqueeze(0)   # shape (1, D)
                mean_vec = mean_feat.flatten().unsqueeze(0)   # shape (1, D)
                
                sim = cosine_similarity(test_vec, mean_vec)   # shape (1, 1)
                feature_similarities.append(sim.item())

                # sim = cosine_similarity(test_feat / test_feat.norm(), mean_feat / mean_feat.norm(), dim=1)
                # feature_similarities.append(sim.item())

        # 5️⃣ KNORA-E
        if knorae: 
            selected_indices = []
            required_correct = k
            while required_correct >= 1 and not selected_indices:
                selected_indices = [i for i, c in enumerate(correct_counts) if c >= required_correct]
                required_correct -= 1
    
            if not selected_indices:
                selected_indices = list(range(len(self.pool)))

            for i in range(len(self.pool)): 
                if i not in selected_indices: 
                    competences[i] = 0.0 
    
        # Step 5: Combine competence & similarity
        if use_sim:
            selected_feature_sims = [s if s > sim_threshold else 0.0 for s in feature_similarities]
            combined_scores = [alpha * c + (1 - alpha) * s for c, s in zip(competences, selected_feature_sims)]
        else:
            combined_scores = competences[:]
        
        if use_fire:
            combined_scores = [s if passed_fire[i] else 0.0 for i, s in enumerate(combined_scores)]
        
        # Step 6: Select models
        if top:
            top_n_idx = np.argsort(combined_scores)[::-1][:n]
            total_score = sum(combined_scores[i] for i in top_n_idx)
            if total_score == 0:
                weights = [1.0 / n] * n
            else:
                weights = [combined_scores[i] / total_score for i in top_n_idx]
        else:
            total_score = sum(combined_scores)
            if total_score == 0:
                weights = [1.0 / len(self.pool)] * len(self.pool)
            else:
                weights = [s / total_score for s in combined_scores]
    
        # Step 7: Weighted aggregation
        num_classes = 1000
        weighted_logits = torch.zeros(num_classes).to(device)

        if top: 
            for idx, weight in zip(top_n_idx, weights):
                weighted_logits += weight * soft_outputs[idx]
        else: 
            for prob, weight in zip(soft_outputs, weights):
                weighted_logits += weight * prob
            
        # Step 8: Suspected attacked model
        min_sim_idx = int(np.argmin(feature_similarities))
        self.suspected_model_votes.append(min_sim_idx)

        if return_logits:
            return weighted_logits
        return weighted_logits.argmax().item()


    def get_top_n_competent_models(self, test_img, k=7, top_n=3, use_sim=False, sim_threshold=0, alpha=0.6):
        # Step 1: Get DINO CLS embedding for the test image
        img_for_dino = test_img.unsqueeze(0).to(device)

        with torch.no_grad():
            test_emb = self.dino_model.forward_features(img_for_dino).cpu().numpy().astype('float32')
            test_emb = test_emb[:, 0, :]  # CLS token only
    
        # Step 2: Find k nearest neighbors in FAISS
        distances, neighbors = self.index.search(test_emb, k)
        neighbor_idxs = neighbors[0]
        local_labels = np.array(self.dsel_labels[neighbor_idxs]).flatten()

        print(local_labels)
    
        # Step 3: Get RoC images
        with torch.no_grad():
            roc_imgs = torch.stack([self.dsel_dataset[idx][0] for idx in neighbor_idxs]).to(device)
    
        # Step 4: Evaluate classifiers
        competences, feature_similarities, correct_counts, grad_vectors  = [], [], [], []
    
        for clf in self.pool:
            clf.eval()
            with torch.no_grad():
                outputs = clf(roc_imgs)
                preds = outputs.argmax(dim=1).cpu().numpy()
                correct = (preds == local_labels).sum()
                competences.append(correct / k)
                correct_counts.append(correct)
    
                # Feature similarity
                test_feat = get_features_before_last_linear(clf, test_img.unsqueeze(0).to(device))
                roc_feats = get_features_before_last_linear(clf, roc_imgs)
                mean_feat = roc_feats.mean(dim=0, keepdim=True)
                sim = cosine_similarity(test_feat.flatten().unsqueeze(0), mean_feat.flatten().unsqueeze(0))
                feature_similarities.append(sim.item())

            test_img_req = test_img.clone().detach().unsqueeze(0).to(device)
            test_img_req.requires_grad_(True)

            out = clf(test_img_req)
            pseudo_label = out.argmax(dim=1)  # keep grad path alive
            loss = F.cross_entropy(out, pseudo_label)
            grad = torch.autograd.grad(loss, test_img_req)[0]
            grad_vec = grad.flatten()  # flatten for cosine similarity
            grad_vectors.append(grad_vec.cpu())


        # Step 5: Compute gradient-based diversity
        grad_vectors_tensor = torch.stack(grad_vectors).to(device)  # (num_models, D)
        num_models = grad_vectors_tensor.size(0)
        
        # Compute cosine similarity matrix between gradient vectors
        cos_sim_matrix = F.cosine_similarity(
            grad_vectors_tensor.unsqueeze(1),  # (num_models, 1, D)
            grad_vectors_tensor.unsqueeze(0),  # (1, num_models, D)
            dim=-1
        )  # (num_models, num_models)
        
        # Compute "diversity score" per model: lower average similarity → more diverse
        diversity_scores = cos_sim_matrix.mean(dim=1).cpu().numpy()  # (num_models,)

        print("diversity_scores: ", diversity_scores)
        print("competences: ", competences)

        final_scores = [alpha * c + (1 - alpha) * d for c, d in zip(competences, diversity_scores)] 
    
        # Step 7: Select top_n models
        top_indices = np.argsort(final_scores)[::-1][:top_n]
        top_models = [self.pool[i] for i in top_indices]

        print(top_indices)
    
        return top_models

### Run

In [41]:
des_model = VisionDES_2(val_dataset, None)
# des_model.fit()

In [42]:
# saved_index = des_model.index
# saved_dsel_embeddings = des_model.dsel_embeddings 
# saved_dsel_labels = des_model.dsel_labels

des_model.index = saved_index 
des_model.dsel_embeddings = saved_dsel_embeddings 
des_model.dsel_labels = saved_dsel_labels

In [43]:
des_model.pool = ens_models  

### DEAA -> SVRE

In [27]:
ens_models = [
    get_models("imagenet", "resnet18", "resnet18"), 
    get_models("imagenet", "deit_tiny_patch16_224", "deit_t"),
    get_models("imagenet", "inception_v3", "inc_v3"), 
    get_models("imagenet", "vit_tiny_patch16_224", "vit_t"), 
    get_models("imagenet", "efficientnet_b0", "efficientnet_b0"), 
    get_models("imagenet", "swin_tiny_patch4_window7_224", "swin_t"), 
    get_models("imagenet", "xcit_tiny_12_p8_224", "swin_t"), 
    get_models("imagenet", "regnetv_040", "swin_t"), 
] 

des_model.pool = ens_models  

In [51]:
adv_images = []     
adv_labels = []

eps = 8/255
alpha = 2/255
iters = 10


for data, label in tqdm(test_loader):
    data, label = data.to(device), label.to(device)

    # 🙏 Please work 
    selected_models = des_model.get_top_n_competent_models(
            data[0],                    
            k=7,
            top_n=4,
            use_sim=False,
            sim_threshold=0,
            alpha=0.1
        )

    adv = svre_mi_fgsm(selected_models, data, label, eps=eps, clip_min=0.0, clip_max=1.0)
    
    adv_images.append(adv.detach().cpu())
    adv_labels.append(label.detach().cpu())

  0%|                                                                                                                                                          | 0/200 [00:00<?, ?it/s]

[652 759 691 413 652 513 413]
diversity_scores:  [0.13200845 0.1337174  0.13681957 0.1276601  0.13932364 0.13100709
 0.13902466 0.13633741]
competences:  [0.42857142857142855, 0.7142857142857143, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571]
[4 6 2 7]


  0%|▋                                                                                                                                                 | 1/200 [00:08<29:02,  8.76s/it]

[978   4 833 977 460 731 625]
diversity_scores:  [0.12961423 0.12725139 0.12931402 0.1251471  0.13461792 0.12490009
 0.12844041 0.12666324]
competences:  [0.2857142857142857, 0.2857142857142857, 0.2857142857142857, 0.42857142857142855, 0.42857142857142855, 0.5714285714285714, 0.5714285714285714, 0.7142857142857143]
[7 6 5 4]


  1%|█▍                                                                                                                                                | 2/200 [00:16<26:03,  7.90s/it]

...

 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌ | 198/200 [17:26<00:12,  6.16s/it]

[737 440 440 737 440 455 440]
diversity_scores:  [0.13037726 0.13291875 0.1317408  0.13811786 0.14084844 0.12559444
 0.12745553 0.12874398]
competences:  [0.7142857142857143, 0.8571428571428571, 0.7142857142857143, 0.7142857142857143, 0.8571428571428571, 0.8571428571428571, 0.8571428571428571, 1.0]
[7 4 1 6]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎| 199/200 [17:32<00:06,  6.10s/it]

[894 894 894 495 894 894 453]
diversity_scores:  [0.13904557 0.1329619  0.12597908 0.13351475 0.13315134 0.12708095
 0.12586087 0.12352893]
competences:  [0.8571428571428571, 0.8571428571428571, 0.5714285714285714, 1.0, 1.0, 0.8571428571428571, 1.0, 0.8571428571428571]
[3 4 6 0]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [17:38<00:00,  5.29s/it]


In [ ]:
ens_models = [
    get_models("imagenet", "resnet18", "resnet18"), 
    get_models("imagenet", "deit_tiny_patch16_224", "deit_t"),
    get_models("imagenet", "inception_v3", "inc_v3"), 
    get_models("imagenet", "vit_tiny_patch16_224", "vit_t"), 
    get_models("imagenet", "efficientnet_b0", "efficientnet_b0"), 
    get_models("imagenet", "swin_tiny_patch4_window7_224", "swin_t"), 
    get_models("imagenet", "xcit_tiny_12_p8_224", "swin_t"), 
    get_models("imagenet", "regnetv_040", "swin_t"), 
]

In [67]:
target_models_2 = [
    get_models("imagenet", "mixer_b16_224", "vit_t"), 
    get_models("imagenet", "convmixer_768_32", "vit_t"),
]

model.safetensors:   0%|          | 0.00/84.9M [00:00<?, ?B/s]

In [68]:
for model in target_models_2:
    model.eval().to(device)
    correct, total = 0, 0
    with torch.no_grad():
        for adv_img, label in tqdm(zip(adv_images, adv_labels), 
                                   total=len(adv_images),
                                   desc=f"Evaluating {model.__class__.__name__} on adv"):
            adv_img = adv_img.to(device)
            label = label.to(device)

            if len(adv_img.size()) == 3: 
                outputs = model(adv_img.unsqueeze(0))
            if len(adv_img.size()) == 4: 
                outputs = model(adv_img) 
                
            if isinstance(outputs, tuple):  # handle models like Inception
                outputs = outputs[0]
            
            preds = outputs.argmax(dim=1)
            correct += (preds == label).sum().item()
            total += label.size(0)
    
    acc = 100 * correct / total
    print(f"{model.__class__.__name__} Accuracy on adv images: {acc:.2f}%")

Evaluating Sequential on adv: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:01<00:00, 146.32it/s]


Sequential Accuracy on adv images: 37.50%


Evaluating Sequential on adv: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:01<00:00, 112.98it/s]

Sequential Accuracy on adv images: 45.50%


In [69]:
for model in target_models_2:
    correct, total = 0, 0
    with torch.no_grad():
        for data, label in tqdm(test_loader, desc=f"Evaluating {model.__class__.__name__}"):
            data, label = data.to(device), label.to(device)
            outputs = model(data)
            if isinstance(outputs, tuple):  # e.g., Inception
                outputs = outputs[0]
            preds = outputs.argmax(dim=1)
            correct += (preds == label).sum().item()
            total += label.size(0)
    acc = 100 * correct / total
    print(f"{model.__class__.__name__} Accuracy: {acc:.2f}%")

Evaluating Sequential: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:03<00:00, 51.07it/s]


Sequential Accuracy: 77.00%


Evaluating Sequential: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:04<00:00, 49.76it/s]

Sequential Accuracy: 85.50%


In [65]:
timm.list_models('*gfnet*')

[]